In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import scipy.stats as sps
import pandas as pd


red = '#FF3300'
blue = '#0099CC'
green = '#00CC66'

sns.set_style("whitegrid")

## Выборочные квантили

### Загрузка данных

В данной задаче вы изучите структуру графика `boxplot` и познакомитесь с выборочными квантилями.

Загрузите данные экспрессии белков у белых мышей и оставьте следующие белки для анализа:
* `pCAMKII_N`
* `pELK_N`
* `ARC_N`

In [ ]:
data = pd.read_csv('Data_Cortex_Nuclear_quantilie.csv')
data.head()

### Подсчет выборочных квантилей

Для посчета выборочных квантилей в библиотеке `numpy` есть функция `numpy.quantile`. На вход ей передается исследуемый массив и значение квантиля.

Посчитайте с помощью этой функции выборочную медиану для признака `pCAMKII_N`

In [ ]:
median = np.nanquantile(data['pCAMKII_N'], 0.5)
print("Выборочная медиана:", median)

Функция должна вернуть `nan`. Проверьте столбец на наличие пропусков, выведите их количество

In [ ]:
missing = data['pCAMKII_N'].isnull().sum()
print("Количество пропусков:", missing)

Получите значение выборочной медианы, удалив пропуски

In [ ]:
median_without_nan = np.nanmedian(data['pCAMKII_N'])
print("Выборочная медиана без пропусков:", median_without_nan)

Для работы с пропущенными значениями в `numpy` есть специальная функция `numpy.nanquantile`

Посчитайте значение выборочной медианы с использованием этой функции (пропуски не удаляйте) и сравните ответы

In [ ]:
median_with_nanquantile = np.nanquantile(data['pCAMKII_N'], 0.5)
print("Выборочная медиана с использованием np.nanquantile:", median_with_nanquantile)

### Визуализация квантилей и изучение boxplot

Допишите функцию для совместной визуализации `boxplot` и гистограммы с нанесенными на нее квантилями

In [ ]:
def draw_quintiles(data, alphas, ymax=0.4):
    '''
    data: pd.Series - столбец из рассматриваемого признака
    alphas: list[float] - массив из значений интересуемых квантилей, которые будут отрисованы
    ymax: float - максимальное значение для вертикальных линий на гистограмме
    '''
    title = data.name
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)},figsize=(14, 8))
    f.suptitle(f"Распределение {title}")

    sns.boxplot(x=data, ax=ax_box, color=blue)

    for alpha in alphas:
        nearest = np.nanquantile(data, alpha)
        ax_hist.vlines(nearest, 0, ymax, color="red", label=f'{alpha} квантиль')
    sns.histplot(data=data, kde=True, ax=ax_hist, stat='density')

    plt.legend()
    plt.show()

Какие квантили соответствуют границам "ящика с усами"? Допишите их и нарисуйте графики для разных признаков

In [ ]:
alphas = [0.25, 0.75]

In [ ]:
draw_quintiles(data.pCAMKII_N, alphas)

In [ ]:
draw_quintiles(data.ARC_N, alphas, 25)

In [ ]:
draw_quintiles(data.pELK_N, alphas, 1.7)

Сделайте вывод

**Ответ:** Из трех графиков видно, что признак pELK_N имеет выбросы. Это видно по кружочкам, которые находятся за границей boxplot и по гистограмме.

###  Сравнение медианы и среднего

Постройте гистограммы для каждой выборки, отметьте на них положение среднего и медианы. Объясните причины полученных расхождений (в случае их наличия)

In [ ]:
plt.figure(figsize=(14,7))
sns.histplot(data['pELK_N'],kde=True)

median_value = np.nanmedian(data['pELK_N'])
mean_value = np.nanmean(data['pELK_N'])

plt.vlines(median_value, 0, 120, color=red,label=r'$\hat{\mu}$')
plt.vlines(mean_value, 0, 120, color='black',label=r'$\overline{X}$')
plt.legend();

**Ответ:** Несовпадение связано как раз-таки с выбросами (ассиметричность данных), т.е. видно наличие длинного правого хвоста. Т.о. можно сденлать вывод о ттом что среднее очень сильно зависит от выбросов.

In [ ]:
plt.figure(figsize=(14,7))
sns.histplot(data['ARC_N'],kde=True)

median_value = np.nanmedian(data['ARC_N'])
mean_value = np.nanmean(data['ARC_N'])

plt.vlines(median_value, 0, 120, color=red,label=r'$\hat{\mu}$')
plt.vlines(mean_value, 0, 120, color='black',label=r'$\overline{X}$')
plt.legend();

**Ответ:** Видно, что распределение данных близко к симметричному, в данном случае распределение похоже на нормальное.

In [ ]:
plt.figure(figsize=(14,7))
sns.histplot(data['pCAMKII_N'],kde=True)
median_value = np.nanmedian(data['pCAMKII_N'])
mean_value = np.nanmean(data['pCAMKII_N'])

plt.vlines(median_value, 0, 120, color=red,label=r'$\hat{\mu}$')
plt.vlines(mean_value, 0, 120, color='black',label=r'$\overline{X}$')
plt.legend();

**Ответ:** Здесь же несовпадение напрямую связано с ассиметричностью данных. Можно сделать вывод о том, что среднее более чувствительно к ассиметричным данным и смещается в сторону более крупных значений.

###  Устранение выбросов

Межквантильным размахом `IQR` называется расстояние между первым и третьем квартилем. Выбросами же принято счиать наблюдения которые находятся за пределами первого или же третьего квантиля на расстоянии более $1.5  \times \text{IQR}$.
Выберете признак с наиболее характерными выбросами, удалите их.

In [ ]:
Q1 = data['pELK_N'].quantile(0.25)
Q3 = data['pELK_N'].quantile(0.75)
IQR = Q3 - Q1
no_outliers = data.pELK_N[(Q1 - 1.5*IQR < data.pELK_N) &  (data.pELK_N < Q3 + 1.5*IQR)]

Выведите гистограмму обработанной выборки и отметьте на нем положения среднего и медианы

In [ ]:
plt.figure(figsize=(14,7))
sns.histplot(no_outliers, kde=True)
median_value = np.median(no_outliers)
mean_value = np.mean(no_outliers)

plt.vlines(median_value, 0, 120, color=red,label=r'$\hat{\mu}$')
plt.vlines(mean_value, 0, 120, color='black',label=r'$\overline{X}$')
plt.xlim(0.6, 2.2)
plt.legend();

А теперь постройте график на неотфильтрованный данных (как в предыдущем пункте). Для наглядности сравнения поставьте соответствующие границы для оси Ox.

In [ ]:
plt.figure(figsize=(14,7))
sns.histplot(data['pELK_N'],kde=True)
median_value = np.nanmedian(data['pELK_N'])
mean_value = np.nanmean(data['pELK_N'])

plt.vlines(median_value, 0, 120, color=red,label=r'$\hat{\mu}$')
plt.vlines(mean_value, 0, 120, color='black',label=r'$\overline{X}$')
plt.xlim(0.6, 2.2)
plt.legend();

Объясните изменение положения оценок

**Ответ:**

1. **До удаления выбросов**:
   - Гистограмма данных `pELK_N` имеет некоторую асимметрию и не является строго нормальным распределением.
   - Медиана отстоит левее, чем среднее значение, и разброс между ними больше. Это может быть вызвано влиянием выбросов, которые смещают среднее вправо.

2. **После удаления выбросов**:
   - Гистограмма обработанной выборки стала ближе к нормальному распределению.
   - Медиана по-прежнему отстоит левее, чем среднее, но разброс между ними уменьшился. Это свидетельствует о том, что удаление выбросов сделало распределение более симметричным и уменьшило влияние выбросов на среднее значение.

**Итог:** Удаление выбросов привело к изменению характеристик распределения данных и повлияло на положение медианы и среднего значения, сделав их более близкими друг к другу.